In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Resnet152


In [3]:
from keras.applications.resnet_v2 import ResNet152V2
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, LearningRateScheduler
from keras.utils import Sequence
import math
import numpy as np

# ResNet152 모델 로드 (최상위 층 제외)
base_model = ResNet152V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모든 층을 훈련 가능하게 설정
for layer in base_model.layers:
    layer.trainable = True

# 새로운 분류기 층 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu', kernel_regularizer='l2')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # 드롭아웃 비율 조정
predictions = Dense(1, activation='sigmoid')(x)

# 새로운 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 옵티마이저 설정 (Adam 사용)
optimizer = Adam(learning_rate=0.0001)

# 모델 컴파일
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

class CombinedDataGenerator(Sequence):
    def __init__(self, generator1, generator2, generator3):
        self.generator1 = generator1
        self.generator2 = generator2
        self.generator3 = generator3
        self.samples = generator1.samples + generator2.samples + generator3.samples
        self.total_len = len(generator1) + len(generator2) + len(generator3)

    def __len__(self):
        return self.total_len

    def __getitem__(self, idx):
        if idx < len(self.generator1):
            return self.generator1[idx]
        idx -= len(self.generator1)

        if idx < len(self.generator2):
            return self.generator2[idx]
        idx -= len(self.generator2)

        return self.generator3[min(idx, len(self.generator3) - 1)]


# ImageDataGenerator 인스턴스 생성
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# 트레이닝 데이터셋 로드 및 설정
train_generator1 = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER3_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

train_generator2 = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER1_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

train_generator3 = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER2_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

# 검증 데이터셋 로드 및 설정
validation_generator1 = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER3_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

validation_generator2 = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER1_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

validation_generator3 = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER2_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

combined_train_generator = CombinedDataGenerator(train_generator1, train_generator2, train_generator3)
combined_validation_generator = CombinedDataGenerator(validation_generator1, validation_generator2, validation_generator3)

# 코사인 니일링 웜업 스케줄러 함수 정의
def cosine_annealing_warmup(epoch, max_epoch=20, base_lr=0.0001, min_lr=0.00001):
    if epoch < 5:  # 웜업 기간 설정 (예: 처음 5 에포크)
        return base_lr * epoch / 5
    else:
        return min_lr + (base_lr - min_lr) * (1 + math.cos(math.pi * (epoch - 5) / (max_epoch - 5))) / 2

# 코사인 니일링 웜업 스케줄러 콜백 생성
cosine_lr_scheduler = LearningRateScheduler(cosine_annealing_warmup)

# 조기 중단 콜백 설정 (정확도 기반)
early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10)

# steps_per_epoch와 validation_steps 계산
steps_per_epoch = combined_train_generator.samples // train_generator1.batch_size
if combined_train_generator.samples % train_generator1.batch_size > 0:
    steps_per_epoch += 1

validation_steps = combined_validation_generator.samples // validation_generator1.batch_size
if combined_validation_generator.samples % validation_generator1.batch_size > 0:
    validation_steps += 1

# 모델 학습
history = model.fit(
    combined_train_generator,
    steps_per_epoch=len(combined_train_generator),
    epochs=20,
    validation_data=combined_validation_generator,
    validation_steps=len(combined_validation_generator),
    callbacks=[cosine_lr_scheduler, early_stopping])

# 모델 저장
model.save('Resnet152_drunk_or_sober.h5')

# 각 epoch에서의 검증 정확도(val_accuracy) 리스트 가져오기
val_accuracies = history.history['val_accuracy']

# 평균 검증 정확도 계산
average_val_accuracy = np.mean(val_accuracies)

print("Average Validation Accuracy:", average_val_accuracy)

234545216/234545216 [==============================] - 8s 0us/step
Found 1502 images belonging to 2 classes.
Found 1493 images belonging to 2 classes.
Found 220 images belonging to 2 classes.
Found 464 images belonging to 2 classes.
Found 323 images belonging to 2 classes.
Found 55 images belonging to 2 classes.
Epoch 1/20
202/202 [==============================] - 1731s 8s/step - loss: 5.5808 - accuracy: 0.4812 - val_loss: 5.2787 - val_accuracy: 0.5570 - lr: 0.0000e+00
Epoch 2/20
202/202 [==============================] - 35s 173ms/step - loss: 4.9775 - accuracy: 0.7453 - val_loss: 4.8480 - val_accuracy: 0.7565 - lr: 2.0000e-05
Epoch 3/20
202/202 [==============================] - 35s 173ms/step - loss: 4.2636 - accuracy: 0.9465 - val_loss: 4.3441 - val_accuracy: 0.8159 - lr: 4.0000e-05
Epoch 4/20
202/202 [==============================] - 35s 173ms/step - loss: 3.7385 - accuracy: 0.9795 - val_loss: 4.0217 - val_accuracy: 0.7779 - lr: 6.0000e-05
Epoch 5/20
202/202 [===================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Average Validation Accuracy: 0.8374703168869019


## Mobilenet_V3

In [4]:
from keras.applications.densenet import DenseNet201
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, LearningRateScheduler
import math
import numpy as np
from keras.applications import MobileNetV3Large

# MobileNetV3Large 모델 로드 (최상위 층 제외)
base_model = MobileNetV3Large(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모든 층을 훈련 가능하게 설정
for layer in base_model.layers:
    layer.trainable = True

# 새로운 분류기 층 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu', kernel_regularizer='l2')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # 드롭아웃 비율 조정
predictions = Dense(1, activation='sigmoid')(x)

# 새로운 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 옵티마이저 설정 (Adam 사용)
optimizer = Adam(learning_rate=0.0001)

# 모델 컴파일
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

class CombinedDataGenerator(Sequence):
    def __init__(self, generator1, generator2, generator3):
        self.generator1 = generator1
        self.generator2 = generator2
        self.generator3 = generator3
        self.samples = generator1.samples + generator2.samples + generator3.samples
        self.total_len = len(generator1) + len(generator2) + len(generator3)

    def __len__(self):
        return self.total_len

    def __getitem__(self, idx):
        if idx < len(self.generator1):
            return self.generator1[idx]
        idx -= len(self.generator1)

        if idx < len(self.generator2):
            return self.generator2[idx]
        idx -= len(self.generator2)

        return self.generator3[min(idx, len(self.generator3) - 1)]


# ImageDataGenerator 인스턴스 생성
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# 트레이닝 데이터셋 로드 및 설정
train_generator1 = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER3_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

train_generator2 = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER1_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

train_generator3 = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER2_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

# 검증 데이터셋 로드 및 설정
validation_generator1 = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER3_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

validation_generator2 = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER1_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

validation_generator3 = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER2_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

combined_train_generator = CombinedDataGenerator(train_generator1, train_generator2, train_generator3)
combined_validation_generator = CombinedDataGenerator(validation_generator1, validation_generator2, validation_generator3)

# 코사인 니일링 웜업 스케줄러 함수 정의
def cosine_annealing_warmup(epoch, max_epoch=20, base_lr=0.0001, min_lr=0.00001):
    if epoch < 5:  # 웜업 기간 설정 (예: 처음 5 에포크)
        return base_lr * epoch / 5
    else:
        return min_lr + (base_lr - min_lr) * (1 + math.cos(math.pi * (epoch - 5) / (max_epoch - 5))) / 2

# 코사인 니일링 웜업 스케줄러 콜백 생성
cosine_lr_scheduler = LearningRateScheduler(cosine_annealing_warmup)

# 조기 중단 콜백 설정 (정확도 기반)
early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10)

# steps_per_epoch와 validation_steps 계산
steps_per_epoch = combined_train_generator.samples // train_generator1.batch_size
if combined_train_generator.samples % train_generator1.batch_size > 0:
    steps_per_epoch += 1

validation_steps = combined_validation_generator.samples // validation_generator1.batch_size
if combined_validation_generator.samples % validation_generator1.batch_size > 0:
    validation_steps += 1

# 모델 학습
history = model.fit(
    combined_train_generator,
    steps_per_epoch=len(combined_train_generator),
    epochs=20,
    validation_data=combined_validation_generator,
    validation_steps=len(combined_validation_generator),
    callbacks=[cosine_lr_scheduler, early_stopping])

# 모델 저장
model.save('Mobilenet_v3_drunk_or_sober.h5')

# 각 epoch에서의 검증 정확도(val_accuracy) 리스트 가져오기
val_accuracies = history.history['val_accuracy']

# 평균 검증 정확도 계산
average_val_accuracy = np.mean(val_accuracies)

print("Average Validation Accuracy:", average_val_accuracy)

12683000/12683000 [==============================] - 1s 0us/step
Found 1502 images belonging to 2 classes.
Found 1493 images belonging to 2 classes.
Found 220 images belonging to 2 classes.
Found 464 images belonging to 2 classes.
Found 323 images belonging to 2 classes.
Found 55 images belonging to 2 classes.
Epoch 1/20
202/202 [==============================] - 39s 69ms/step - loss: 5.1049 - accuracy: 0.4896 - val_loss: 4.8308 - val_accuracy: 0.4596 - lr: 0.0000e+00
Epoch 2/20
202/202 [==============================] - 11s 55ms/step - loss: 4.6918 - accuracy: 0.6675 - val_loss: 4.9666 - val_accuracy: 0.4596 - lr: 2.0000e-05
Epoch 3/20
202/202 [==============================] - 11s 54ms/step - loss: 4.2369 - accuracy: 0.8473 - val_loss: 5.1279 - val_accuracy: 0.4596 - lr: 4.0000e-05
Epoch 4/20
202/202 [==============================] - 11s 55ms/step - loss: 3.8931 - accuracy: 0.9210 - val_loss: 4.3143 - val_accuracy: 0.4691 - lr: 6.0000e-05
Epoch 5/20
202/202 [========================

## Inception_V3

In [5]:
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, LearningRateScheduler
import math
import numpy as np

# InceptionV3 모델 로드 (최상위 층 제외)
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))  # InceptionV3는 299x299 입력을 사용

# 모든 층을 훈련 가능하게 설정
for layer in base_model.layers:
    layer.trainable = True

# 새로운 분류기 층 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu', kernel_regularizer='l2')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # 드롭아웃 비율 조정
predictions = Dense(1, activation='sigmoid')(x)

# 새로운 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 옵티마이저 설정 (Adam 사용)
optimizer = Adam(learning_rate=0.0001)

# 모델 컴파일
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

class CombinedDataGenerator(Sequence):
    def __init__(self, generator1, generator2, generator3):
        self.generator1 = generator1
        self.generator2 = generator2
        self.generator3 = generator3
        self.samples = generator1.samples + generator2.samples + generator3.samples
        self.total_len = len(generator1) + len(generator2) + len(generator3)

    def __len__(self):
        return self.total_len

    def __getitem__(self, idx):
        if idx < len(self.generator1):
            return self.generator1[idx]
        idx -= len(self.generator1)

        if idx < len(self.generator2):
            return self.generator2[idx]
        idx -= len(self.generator2)

        return self.generator3[min(idx, len(self.generator3) - 1)]


# ImageDataGenerator 인스턴스 생성
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# 트레이닝 데이터셋 로드 및 설정
train_generator1 = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER3_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

train_generator2 = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER1_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

train_generator3 = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER2_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

# 검증 데이터셋 로드 및 설정
validation_generator1 = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER3_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

validation_generator2 = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER1_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

validation_generator3 = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER2_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

combined_train_generator = CombinedDataGenerator(train_generator1, train_generator2, train_generator3)
combined_validation_generator = CombinedDataGenerator(validation_generator1, validation_generator2, validation_generator3)

# 코사인 니일링 웜업 스케줄러 함수 정의
def cosine_annealing_warmup(epoch, max_epoch=20, base_lr=0.0001, min_lr=0.00001):
    if epoch < 5:  # 웜업 기간 설정 (예: 처음 5 에포크)
        return base_lr * epoch / 5
    else:
        return min_lr + (base_lr - min_lr) * (1 + math.cos(math.pi * (epoch - 5) / (max_epoch - 5))) / 2

# 코사인 니일링 웜업 스케줄러 콜백 생성
cosine_lr_scheduler = LearningRateScheduler(cosine_annealing_warmup)

# 조기 중단 콜백 설정 (정확도 기반)
early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10)

# steps_per_epoch와 validation_steps 계산
steps_per_epoch = combined_train_generator.samples // train_generator1.batch_size
if combined_train_generator.samples % train_generator1.batch_size > 0:
    steps_per_epoch += 1

validation_steps = combined_validation_generator.samples // validation_generator1.batch_size
if combined_validation_generator.samples % validation_generator1.batch_size > 0:
    validation_steps += 1

# 모델 학습
history = model.fit(
    combined_train_generator,
    steps_per_epoch=len(combined_train_generator),
    epochs=20,
    validation_data=combined_validation_generator,
    validation_steps=len(combined_validation_generator),
    callbacks=[cosine_lr_scheduler, early_stopping])

# 모델 저장
model.save('InceptionV3_drunk_or_sober.h5')

# 각 epoch에서의 검증 정확도(val_accuracy) 리스트 가져오기
val_accuracies = history.history['val_accuracy']

# 평균 검증 정확도 계산
average_val_accuracy = np.mean(val_accuracies)

print("Average Validation Accuracy:", average_val_accuracy)

87910968/87910968 [==============================] - 4s 0us/step
Found 1502 images belonging to 2 classes.
Found 1493 images belonging to 2 classes.
Found 220 images belonging to 2 classes.
Found 464 images belonging to 2 classes.
Found 323 images belonging to 2 classes.
Found 55 images belonging to 2 classes.
Epoch 1/20
202/202 [==============================] - 49s 98ms/step - loss: 5.4577 - accuracy: 0.5580 - val_loss: 5.3855 - val_accuracy: 0.4489 - lr: 0.0000e+00
Epoch 2/20
202/202 [==============================] - 15s 72ms/step - loss: 5.0634 - accuracy: 0.7474 - val_loss: 5.1456 - val_accuracy: 0.6651 - lr: 2.0000e-05
Epoch 3/20
202/202 [==============================] - 15s 73ms/step - loss: 4.4613 - accuracy: 0.9524 - val_loss: 4.7921 - val_accuracy: 0.7447 - lr: 4.0000e-05
Epoch 4/20
202/202 [==============================] - 15s 73ms/step - loss: 4.1028 - accuracy: 0.9857 - val_loss: 4.4044 - val_accuracy: 0.8100 - lr: 6.0000e-05
Epoch 5/20
202/202 [========================

## DenseNet201

In [6]:
from keras.applications.densenet import DenseNet201
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, LearningRateScheduler
import math
import numpy as np

# DenseNet201 모델 로드 (최상위 층 제외)
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모든 층을 훈련 가능하게 설정
for layer in base_model.layers:
    layer.trainable = True

# 새로운 분류기 층 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu', kernel_regularizer='l2')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # 드롭아웃 비율 조정
predictions = Dense(1, activation='sigmoid')(x)

# 새로운 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 옵티마이저 설정 (Adam 사용)
optimizer = Adam(learning_rate=0.0001)

# 모델 컴파일
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

class CombinedDataGenerator(Sequence):
    def __init__(self, generator1, generator2, generator3):
        self.generator1 = generator1
        self.generator2 = generator2
        self.generator3 = generator3
        self.samples = generator1.samples + generator2.samples + generator3.samples
        self.total_len = len(generator1) + len(generator2) + len(generator3)

    def __len__(self):
        return self.total_len

    def __getitem__(self, idx):
        if idx < len(self.generator1):
            return self.generator1[idx]
        idx -= len(self.generator1)

        if idx < len(self.generator2):
            return self.generator2[idx]
        idx -= len(self.generator2)

        return self.generator3[min(idx, len(self.generator3) - 1)]


# ImageDataGenerator 인스턴스 생성
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# 트레이닝 데이터셋 로드 및 설정
train_generator1 = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER3_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

train_generator2 = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER1_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

train_generator3 = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER2_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

# 검증 데이터셋 로드 및 설정
validation_generator1 = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER3_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

validation_generator2 = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER1_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

validation_generator3 = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER2_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

combined_train_generator = CombinedDataGenerator(train_generator1, train_generator2, train_generator3)
combined_validation_generator = CombinedDataGenerator(validation_generator1, validation_generator2, validation_generator3)

# 코사인 니일링 웜업 스케줄러 함수 정의
def cosine_annealing_warmup(epoch, max_epoch=20, base_lr=0.0001, min_lr=0.00001):
    if epoch < 5:  # 웜업 기간 설정 (예: 처음 5 에포크)
        return base_lr * epoch / 5
    else:
        return min_lr + (base_lr - min_lr) * (1 + math.cos(math.pi * (epoch - 5) / (max_epoch - 5))) / 2

# 코사인 니일링 웜업 스케줄러 콜백 생성
cosine_lr_scheduler = LearningRateScheduler(cosine_annealing_warmup)

# 조기 중단 콜백 설정 (정확도 기반)
early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10)

# steps_per_epoch와 validation_steps 계산
steps_per_epoch = combined_train_generator.samples // train_generator1.batch_size
if combined_train_generator.samples % train_generator1.batch_size > 0:
    steps_per_epoch += 1

validation_steps = combined_validation_generator.samples // validation_generator1.batch_size
if combined_validation_generator.samples % validation_generator1.batch_size > 0:
    validation_steps += 1

# 모델 학습
history = model.fit(
    combined_train_generator,
    steps_per_epoch=len(combined_train_generator),
    epochs=20,
    validation_data=combined_validation_generator,
    validation_steps=len(combined_validation_generator),
    callbacks=[cosine_lr_scheduler, early_stopping])

# 모델 저장
model.save('DenseNet201_drunk_or_sober.h5')

# 각 epoch에서의 검증 정확도(val_accuracy) 리스트 가져오기
val_accuracies = history.history['val_accuracy']

# 평균 검증 정확도 계산
average_val_accuracy = np.mean(val_accuracies)

print("Average Validation Accuracy:", average_val_accuracy)

74836368/74836368 [==============================] - 3s 0us/step
Found 1502 images belonging to 2 classes.
Found 1493 images belonging to 2 classes.
Found 220 images belonging to 2 classes.
Found 464 images belonging to 2 classes.
Found 323 images belonging to 2 classes.
Found 55 images belonging to 2 classes.
Epoch 1/20
202/202 [==============================] - 132s 202ms/step - loss: 5.5718 - accuracy: 0.4855 - val_loss: 5.7090 - val_accuracy: 0.5297 - lr: 0.0000e+00
Epoch 2/20
202/202 [==============================] - 30s 147ms/step - loss: 4.6692 - accuracy: 0.8022 - val_loss: 4.5563 - val_accuracy: 0.7399 - lr: 2.0000e-05
Epoch 3/20
202/202 [==============================] - 30s 146ms/step - loss: 3.6558 - accuracy: 0.9639 - val_loss: 3.7582 - val_accuracy: 0.7755 - lr: 4.0000e-05
Epoch 4/20
202/202 [==============================] - 30s 147ms/step - loss: 2.8022 - accuracy: 0.9925 - val_loss: 2.8407 - val_accuracy: 0.8610 - lr: 6.0000e-05
Epoch 5/20
202/202 [===================

## EffcientB7

In [7]:
from keras.applications.efficientnet import EfficientNetB7
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, LearningRateScheduler
import math
import numpy as np

# EfficientNetB0 모델 로드 (최상위 층 제외)
base_model = EfficientNetB7(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모든 층을 훈련 가능하게 설정
for layer in base_model.layers:
    layer.trainable = True

# 새로운 분류기 층 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu', kernel_regularizer='l2')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # 드롭아웃 비율 조정
predictions = Dense(1, activation='sigmoid')(x)

# 새로운 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 옵티마이저 설정 (Adam 사용)
optimizer = Adam(learning_rate=0.0001)

# 모델 컴파일
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

class CombinedDataGenerator(Sequence):
    def __init__(self, generator1, generator2, generator3):
        self.generator1 = generator1
        self.generator2 = generator2
        self.generator3 = generator3
        self.samples = generator1.samples + generator2.samples + generator3.samples
        self.total_len = len(generator1) + len(generator2) + len(generator3)

    def __len__(self):
        return self.total_len

    def __getitem__(self, idx):
        if idx < len(self.generator1):
            return self.generator1[idx]
        idx -= len(self.generator1)

        if idx < len(self.generator2):
            return self.generator2[idx]
        idx -= len(self.generator2)

        return self.generator3[min(idx, len(self.generator3) - 1)]


# ImageDataGenerator 인스턴스 생성
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# 트레이닝 데이터셋 로드 및 설정
train_generator1 = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER3_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

train_generator2 = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER1_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

train_generator3 = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER2_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

# 검증 데이터셋 로드 및 설정
validation_generator1 = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER3_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

validation_generator2 = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER1_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

validation_generator3 = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER2_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

combined_train_generator = CombinedDataGenerator(train_generator1, train_generator2, train_generator3)
combined_validation_generator = CombinedDataGenerator(validation_generator1, validation_generator2, validation_generator3)

# 코사인 니일링 웜업 스케줄러 함수 정의
def cosine_annealing_warmup(epoch, max_epoch=20, base_lr=0.0001, min_lr=0.00001):
    if epoch < 5:  # 웜업 기간 설정 (예: 처음 5 에포크)
        return base_lr * epoch / 5
    else:
        return min_lr + (base_lr - min_lr) * (1 + math.cos(math.pi * (epoch - 5) / (max_epoch - 5))) / 2

# 코사인 니일링 웜업 스케줄러 콜백 생성
cosine_lr_scheduler = LearningRateScheduler(cosine_annealing_warmup)

# 조기 중단 콜백 설정 (정확도 기반)
early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10)

# steps_per_epoch와 validation_steps 계산
steps_per_epoch = combined_train_generator.samples // train_generator1.batch_size
if combined_train_generator.samples % train_generator1.batch_size > 0:
    steps_per_epoch += 1

validation_steps = combined_validation_generator.samples // validation_generator1.batch_size
if combined_validation_generator.samples % validation_generator1.batch_size > 0:
    validation_steps += 1

# 모델 학습
history = model.fit(
    combined_train_generator,
    steps_per_epoch=len(combined_train_generator),
    epochs=20,
    validation_data=combined_validation_generator,
    validation_steps=len(combined_validation_generator),
    callbacks=[cosine_lr_scheduler, early_stopping])

# 모델 저장
model.save('EfficientNetB7_drunk_or_sober.h5')

# 각 epoch에서의 검증 정확도(val_accuracy) 리스트 가져오기
val_accuracies = history.history['val_accuracy']

# 평균 검증 정확도 계산
average_val_accuracy = np.mean(val_accuracies)

print("Average Validation Accuracy:", average_val_accuracy)

258076736/258076736 [==============================] - 8s 0us/step
Found 1502 images belonging to 2 classes.
Found 1493 images belonging to 2 classes.
Found 220 images belonging to 2 classes.
Found 464 images belonging to 2 classes.
Found 323 images belonging to 2 classes.
Found 55 images belonging to 2 classes.
Epoch 1/20
202/202 [==============================] - 180s 343ms/step - loss: 5.7494 - accuracy: 0.4942 - val_loss: 5.3620 - val_accuracy: 0.4739 - lr: 0.0000e+00
Epoch 2/20
202/202 [==============================] - 64s 314ms/step - loss: 5.1841 - accuracy: 0.7425 - val_loss: 5.1758 - val_accuracy: 0.6853 - lr: 2.0000e-05
Epoch 3/20
202/202 [==============================] - 64s 314ms/step - loss: 4.6577 - accuracy: 0.9166 - val_loss: 4.8251 - val_accuracy: 0.7803 - lr: 4.0000e-05
Epoch 4/20
202/202 [==============================] - 63s 314ms/step - loss: 4.2673 - accuracy: 0.9661 - val_loss: 4.6187 - val_accuracy: 0.7660 - lr: 6.0000e-05
Epoch 5/20
202/202 [=================